## AI서비스엔지니어4기(1주차 2일)
- 스파르타코딩클럽 재직자 대상 교육
- EDA와 전처리

### 01 Library Definition & Hyperparameter Definition
- 일단 google.colab mount부터 해야함

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np

num_sample = 10

missing_value_flag = 'remove' # if flag is 'replace', missing values are replaced with the mode, etc.
scaling_flag = 'min-max' # if flag is 'standardization', numerical values are scaled by standardization

### 02 Data Load & Check Data

In [ ]:
import os

base_path = '/content/drive/MyDrive/AI_serivce_track/Week_01'

# 정확한 이름으로 다시 할당
data_path = os.path.join(base_path, 'Loan_Default.csv')

In [ ]:
eda_df = pd.read_csv(data_path)
eda_df.head(num_sample)

,ID,year,loan_limit,Gender,approv_in_adv,loan_type,loan_purpose,Credit_Worthiness,open_credit,business_or_commercial,...,credit_type,Credit_Score,co-applicant_credit_type,age,submission_of_application,LTV,Region,Security_Type,Status,dtir1
0,24890,2019,cf,Sex Not Available,nopre,type1,p1,l1,nopc,nob/c,...,EXP,758,CIB,25-34,to_inst,98.728814,south,direct,1,45.0
1,24891,2019,cf,Male,nopre,type2,p1,l1,nopc,b/c,...,EQUI,552,EXP,55-64,to_inst,NaN,North,direct,1,NaN
2,24892,2019,cf,Male,pre,type1,p1,l1,nopc,nob/c,...,EXP,834,CIB,35-44,to_inst,80.019685,south,direct,0,46.0
3,24893,2019,cf,Male,nopre,type1,p4,l1,nopc,nob/c,...,EXP,587,CIB,45-54,not_inst,69.376900,North,direct,0,42.0
4,24894,2019,cf,Joint,pre,type1,p1,l1,nopc,nob/c,...,CRIF,602,EXP,25-34,not_inst,91.886544,North,direct,0,39.0
5,24895,2019,cf,Joint,pre,type1,p1,l1,nopc,nob/c,...,EXP,864,EXP,35-44,not_inst,70.089286,North,direct,0,40.0
6,24896,2019,cf,Joint,pre,type1,p3,l1,nopc,nob/c,...,EXP,860,EXP,55-64,to_inst,79.109589,North,direct,0,44.0
7,24897,2019,NaN,Female,nopre,type1,p4,l1,nopc,nob/c,...,CIB,863,CIB,55-64,to_inst,86.525974,North,direct,0,42.0
8,24898,2019,cf,Joint,nopre,type1,p3,l1,nopc,nob/c,...,CIB,580,EXP,55-64,to_inst,78.765690,central,direct,0,44.0
9,24899,2019,cf,Sex Not Available,nopre,type3,p3,l1,nopc,nob/c,...,CIB,788,EXP,55-64,to_inst,63.444767,south,direct,0,30.0


In [ ]:
print('# of data: {}'.format(eda_df.shape[0]))
print('# of features: {}\n'.format(eda_df.shape[1]))

# of data: 148670
# of features: 34



In [ ]:
print('feature map/dtype')
numerical_count = 0
categorical_count = 0

for feature_name, feature_type in zip(eda_df.columns, eda_df.dtypes):
  if np.issubdtype(feature_type, np.number):
    numerical_count += 1
  else:
    categorical_count += 1
  print('{}: {}'.format(feature_name, feature_type))

print('\n # of numerical item: {}/# of categorical item: {}'.format(numerical_count, categorical_count))

feature map/dtype
ID: int64
year: int64
loan_limit: object
Gender: object
approv_in_adv: object
loan_type: object
loan_purpose: object
Credit_Worthiness: object
open_credit: object
business_or_commercial: object
loan_amount: int64
rate_of_interest: float64
Interest_rate_spread: float64
Upfront_charges: float64
term: float64
Neg_ammortization: object
interest_only: object
lump_sum_payment: object
property_value: float64
construction_type: object
occupancy_type: object
Secured_by: object
total_units: object
income: float64
credit_type: object
Credit_Score: int64
co-applicant_credit_type: object
age: object
submission_of_application: object
LTV: float64
Region: object
Security_Type: object
Status: int64
dtir1: float64

 # of numerical item: 13/# of categorical item: 21


In [ ]:
print('\nstatistcal infos')
eda_df.describe()

label_df = eda_df['Status']
label_dict = label_df.value_counts()

print('label shape: {}'.format(label_df.shape))

print('\nnum of negative label: {}/num of positive label: {}'.format(label_dict[0], label_dict[1]))
print('% of negative label: {}/% of positive label: {}'.format(label_dict[0] / label_df.shape[0] * 100, label_dict[1] / label_df.shape[0] * 100))


statistcal infos
label shape: (148670,)

num of negative label: 112031/num of positive label: 36639
% of negative label: 75.3554853030201/% of positive label: 24.64451469697989


### 03 Data cleaning

#### 03-01 중복 제거
- 항상 중복이 있는지 확인

In [ ]:
print('Start Removing Duplicates') # 중복 제거
duplicates = eda_df.duplicated() # .duplicated()함수 -> 중복 구하기
eda_df = eda_df[~duplicates] # 중복제거 실행
print('# of data after removing duplicates:{}\n'.format(eda_df.shape[0]))


Start Removing Duplicates
# of data after removing duplicates:148670



#### 03-02 결측치 체크

In [ ]:
print('Start Handling Missing Values')
print('features / # of missing values')
print(eda_df.isnull().sum()) # .isnull().sum() 이건 결측치의 수 체크!

Start Handling Missing Values
features / # of missing values
ID                               0
year                             0
loan_limit                    3344
Gender                           0
approv_in_adv                  908
loan_type                        0
loan_purpose                   134
Credit_Worthiness                0
open_credit                      0
business_or_commercial           0
loan_amount                      0
rate_of_interest             36439
Interest_rate_spread         36639
Upfront_charges              39642
term                            41
Neg_ammortization              121
interest_only                    0
lump_sum_payment                 0
property_value               15098
construction_type                0
occupancy_type                   0
Secured_by                       0
total_units                      0
income                        9150
credit_type                      0
Credit_Score                     0
co-applicant_credit_type     

In [ ]:
if missing_value_flag == 'remove': # 앞에서 정의한 missing_value_flag가 'remove'인 경우
  print('Remove missing values\n')
  eda_df = eda_df.dropna()
elif missing_value_flag == 'replace': # 앞에서 정의한 missing_value_flag가 'replace'인 경우
  print('Replace missing values with mode\n')
  for feature_name in eda_df.columns :
    if eda_df[feature_name].isnull().sum() > 0:  # missing_value 개수가 1개 이상이면
      eda_df[feature_name].fillna(eda_df[feature_name].mode().iloc[0], inplace=True) # .mode()로 채우기(최빈값)

Remove missing values



In [ ]:
print('Start Handling Outliers using IQR')
outlier_dict = {}

for feature_name in eda_df.columns:
  if np.issubdtype(eda_df[feature_name].dtype, np.number):
    Q1 = eda_df[feature_name].quantile(0.25) # Q1 정의 (하위 25%)
    Q3 = eda_df[feature_name].quantile(0.75) # Q3 정의 (상위 25%)
    IQR = Q3 - Q1 # IQR = Inter Quantile Range (Q3에서 Q1을 뺀 것!)
    lower_bound = Q1 - 1.5 * IQR # 아웃라이어 하한 Q1 - 1.5*IQR
    upper_bound = Q3 + 1.5 * IQR # 아웃라이어 상한 Q3 + 1.5*IQR

    num_outlier = ((eda_df[feature_name] < lower_bound) | (eda_df[feature_name] > upper_bound)).sum() # 아웃라이어 개수 저장
    outlier_dict[feature_name] = num_outlier

    eda_df = eda_df[(eda_df[feature_name] >= lower_bound) & (eda_df[feature_name] <= upper_bound)]


print('Total # of outliers: {}'.format(sum(outlier_dict.values())))
print('Number of outliers for each feature')
for feature_name, num_outlier in outlier_dict.items():
  print('{}: {}'.format(feature_name, num_outlier))

Start Handling Outliers using IQR
Total # of outliers: 29744
Number of outliers for each feature
ID: 0
year: 0
loan_amount: 949
rate_of_interest: 353
Interest_rate_spread: 494
Upfront_charges: 2536
term: 17764
property_value: 2405
income: 2865
Credit_Score: 0
LTV: 1122
Status: 0
dtir1: 1256


##### 위 코드를 좀 바꿔본다면?
- 위의 코드가 직관적이고 편리하게 짜여졌지만 원본을 남겨두는게 안전할 수도 있음

In [ ]:
# eda_df_cleaned = eda_df.copy()
# outlier_dict = {}

# for col in eda_df_cleaned.select_dtypes(include=np.number).columns:
#     Q1 = eda_df_cleaned[col].quantile(0.25)
#     Q3 = eda_df_cleaned[col].quantile(0.75)
#     IQR = Q3 - Q1
#     lower = Q1 - 1.5 * IQR
#     upper = Q3 + 1.5 * IQR

#     outliers = (eda_df_cleaned[col] < lower) | (eda_df_cleaned[col] > upper)
#     outlier_dict[col] = outliers.sum()

#     eda_df_cleaned = eda_df_cleaned[~outliers]

# print(f"총 이상치 수: {sum(outlier_dict.values())}")

### 04. Data Preprocessing
- 데이터 전처리 하기
  - 여기서는 min-max scaling, standard scaling을 다루고 있음

In [ ]:
numerical_features = eda_df.select_dtypes(include = [np.number])
numerical_array = numerical_features.values
print('numerical features shape: {}'.format(numerical_array.shape))

numerical features shape: (68443, 13)


In [ ]:
eda_df.shape

(68443, 34)

In [ ]:

if scaling_flag == 'standardization':
  print('Standardization of numeircal features\n')
  feature_mean = np.mean(numerical_array, axis=0)
  feature_std = np.std(numerical_array, axis=0)

  feature_std = np.where(feature_std == 0, 1e-5, feature_std) # 표준편차가 0일 때 1e-5를 넣어서 처리?

  scaled_data = (numerical_array - feature_mean) / feature_std
elif scaling_flag == 'min-max':
  print('Min-max normalization of numeircal features\n')
  data_min = np.min(numerical_array, axis=0)
  data_max = np.max(numerical_array, axis=0)

  data_max = np.where(data_max == data_min, data_min + 1e-5, data_max) # min/max가 같으면 분모가 0이 되기 때문에

  scaled_data = (numerical_array - data_min) / (data_max - data_min)

categorical_features = eda_df.select_dtypes(include=[object]) # .select_dtypes, 데이터 범주 선택 (object고르기)
categorical_array = categorical_features.values
print('categorical features shape: {}'.format(categorical_array.shape))

categorical_encoded_list = [] # 범주형 컬럼은 encoding을 해야함
print('One-hot encoding of categorical features\n') # 보통 One-hot encoding을 함!
num_encoded_dim = 0
for i, feature_name in enumerate(categorical_features.columns):
    unique_values = np.unique(categorical_array[:, i])
    print('{}: {}'.format(feature_name, unique_values))
    num_encoded_dim += len(unique_values)

    encoded_array = np.zeros((categorical_array.shape[0], len(unique_values)))
    for j, value in enumerate(categorical_array[:, i]):
        encoded_array[j, np.where(unique_values == value)[0]] = 1
    categorical_encoded_list.append(encoded_array)

categorical_encoded_array = np.hstack(categorical_encoded_list)
print('\none-hot encoded dim: {}'.format(num_encoded_dim))
print('categorical encoded array shape: {}'.format(categorical_encoded_array.shape))

processed_array = np.hstack((scaled_data, categorical_encoded_array))
print('processed array shape: {}'.format(processed_array.shape))

Min-max normalization of numeircal features

categorical features shape: (68443, 21)
One-hot encoding of categorical features

loan_limit: ['cf' 'ncf']
Gender: ['Female' 'Joint' 'Male' 'Sex Not Available']
approv_in_adv: ['nopre' 'pre']
loan_type: ['type1' 'type2' 'type3']
loan_purpose: ['p1' 'p2' 'p3' 'p4']
Credit_Worthiness: ['l1' 'l2']
open_credit: ['nopc']
business_or_commercial: ['b/c' 'nob/c']
Neg_ammortization: ['neg_amm' 'not_neg']
interest_only: ['int_only' 'not_int']
lump_sum_payment: ['lpsm' 'not_lpsm']
construction_type: ['sb']
occupancy_type: ['ir' 'pr' 'sr']
Secured_by: ['home']
total_units: ['1U' '2U' '3U' '4U']
credit_type: ['CIB' 'CRIF' 'EQUI' 'EXP']
co-applicant_credit_type: ['CIB' 'EXP']
age: ['25-34' '35-44' '45-54' '55-64' '65-74' '<25' '>74']
submission_of_application: ['not_inst' 'to_inst']
Region: ['North' 'North-East' 'central' 'south']
Security_Type: ['direct']

one-hot encoded dim: 55
categorical encoded array shape: (68443, 55)
processed array shape: (68443,